In [1]:
pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.2 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 5.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from diffusers import DiffusionPipeline
from torchvision import models, transforms
from PIL import Image
import os
from pycocotools.coco import COCO
import numpy as np
from sklearn.svm import SVC
import joblib
import concurrent.futures
from accelerate import Accelerator

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [9]:
# The train2017 and instances_train2017_subset are subsets of COCO's dataset, they contain only 2000 images
coco_root = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
coco_annotation_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json'
coco = COCO(coco_annotation_file)
gan_root = '/kaggle/input/gan-2000-images-64x64' 



loading annotations into memory...
Done (t=26.00s)
creating index...
index created!


In [4]:
# Check if I have a GPU available 
device = "cuda" if torch.cuda.is_available() else "cpu"

dtype = torch.float16 if device == "cuda" else torch.float32

# Inizializza Accelerator per la gestione delle GPU
accelerator = Accelerator()

In [5]:
# Funzione per estrarre caratteristiche con VGG
def extract_vgg_features(image, model):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(image).unsqueeze(0).to(accelerator.device)  # add the dimension batch and transfer to the GPU
    with torch.no_grad():
        features = model(input_tensor)
    return features.flatten().cpu().numpy()
# Load a VGG16 pre-trained model and remove the last layer
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier = vgg16.classifier[:-1]  # remove the last layer
vgg16.eval().to(accelerator.device)  # put in eval modality and transfer to the GPU

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 209MB/s]  


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
def save_image_256(image, save_dir, img_id):
    os.makedirs(save_dir, exist_ok=True)  # Crea la cartella se non esiste
    save_path = os.path.join(save_dir, f"image_256x256{img_id}.jpg")
    image.save(save_path, format="JPEG")
    print(f"Immagine salvata in: {save_path}")
def save_image_64(image, save_dir, img_id):
    os.makedirs(save_dir, exist_ok=True)  # Crea la cartella se non esiste
    save_path = os.path.join(save_dir, f"image_64x64{img_id}.jpg")
    image.save(save_path, format="JPEG")
    print(f"Immagine salvata in: {save_path}")
def save_image_fake(image, save_dir, img_id):
    os.makedirs(save_dir, exist_ok=True)  # Crea la cartella se non esiste
    save_path = os.path.join(save_dir, f"fake_image_{img_id}.jpg")
    image.save(save_path, format="JPEG")
    print(f"Immagine salvata in: {save_path}")
# Funzione per salvare le immagini reali
def save_image_real(image, save_dir, img_id):
    os.makedirs(save_dir, exist_ok=True)  # Crea la cartella se non esiste
    save_path = os.path.join(save_dir, f"real_image_{img_id}.jpg")
    image.save(save_path, format="JPEG")
    print(f"Immagine salvata in: {save_path}")

In [10]:
# Carica il modello di diffusione
pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=dtype)
pipeline.to(accelerator.device)

# Funzione per generare immagini
def generate_image(prompt):
    image = pipeline(prompt, num_inference_steps=8).images[0]
    return image.convert("RGB")


# Directory di salvataggio
real_dir = "/kaggle/working/real_images"
fake_dir = "/kaggle/working/fake_images"
classifier = SVC(kernel='linear')  # Usa tutte le CPU disponibili
features_list = []
labels = []
last_step = 0

# Estrai immagini da COCO
image_ids = coco.getImgIds()
sample_ids = image_ids[last_step:]

step = 0

# Generazione immagini e raccolta caratteristiche tqdm
for img_id in sample_ids:
    img_data = coco.imgs[img_id]
    img_path = os.path.join(coco_root, img_data['file_name'])
    if step >= 2000:  # Limita a 2000 immagini
        break

    try:
        # Immagine reale
        image_real = Image.open(img_path).convert("RGB")
        real_features = extract_vgg_features(image_real, vgg16)
        features_list.append(real_features)
        labels.append(1)  # etichetta per le immagini reali
        save_image_real(image_real, real_dir, img_id)

        # Ottieni categorie per generare immagini fake
        ann_ids = coco.getAnnIds(imgIds=[img_id], iscrowd=False)
        anns = coco.loadAnns(ann_ids)
        categories = [coco.cats[ann['category_id']]['name'] for ann in anns]
        prompt = ", ".join(categories)

        # Generazione immagini in parallelo
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_image = executor.submit(generate_image, prompt)
            image_fake = future_image.result()  # Aspetta il risultato
            save_image_fake(image_fake, fake_dir, img_id)

        fake_features = extract_vgg_features(image_fake, vgg16)
        features_list.append(fake_features)
        labels.append(0)  # etichetta per le immagini fake

        step += 1

    except Exception as e:
        print(f"Errore nell'elaborazione dell'immagine ID {img_id}: {e}")

step = 0
# Elaborazione delle immagini GAN
gan_images = os.listdir(gan_root)
for gan_img_name in gan_images:
    img_path = os.path.join(gan_root, gan_img_name)
    try:
        # Immagine GAN
        gan_image = Image.open(img_path).convert("RGB")
        gan_features = extract_vgg_features(gan_image, vgg16)
        features_list.append(gan_features)
        labels.append(0)  # etichetta per le immagini GAN
        print(step)
        step += 1

    except Exception as e:
        print(f"Errore nell'elaborazione dell'immagine ID {img_id}: {e}")

# Addestra il classificatore SVM
classifier.fit(features_list, labels)

# Salva il modello finale
joblib.dump(classifier, 'svm_classifier.pkl')
print("Modello SVM salvato come 'svm_classifier.pkl'")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Immagine salvata in: /kaggle/working/real_images/real_image_391895.jpg


  0%|          | 0/8 [00:00<?, ?it/s]

Immagine salvata in: /kaggle/working/fake_images/fake_image_391895.jpg
Immagine salvata in: /kaggle/working/real_images/real_image_522418.jpg


  0%|          | 0/8 [00:00<?, ?it/s]

...

Immagine salvata in: /kaggle/working/fake_images/fake_image_184613.jpg
Immagine salvata in: /kaggle/working/real_images/real_image_318219.jpg


  0%|          | 0/8 [00:00<?, ?it/s]

Immagine salvata in: /kaggle/working/fake_images/fake_image_318219.jpg
Immagine salvata in: /kaggle/working/real_images/real_image_554625.jpg


  0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
# The train2017 and instances_train2017_subset are subsets of COCO's dataset, they contain only 2000 images
coco_root = '/kaggle/input/coco-subset'
gan_root = '/kaggle/input/gan-2000-images-64x64' 
stable_root = '/kaggle/input/stable-diffusion'

# Funzione per upsampling a 256x256 se la dimensione è inferiore a 256x256
def upsample_to_256(image):
    if image.size[0] < 256 or image.size[1] < 256:
        return image.resize((256, 256), Image.BICUBIC)
    return image

# Funzione per downsampling a 256x256 se la dimensione è superiore a 256x256
def downsample_to_256(image):
    if image.size[0] > 256 or image.size[1] > 256:
        return image.resize((256, 256), Image.LANCZOS)
    return image
# Funzione per downsampling a 64x64 se la dimensione è superiore a 64x64
def downsample_to_64(image):
    if image.size[0] > 64 or image.size[1] > 64:
        return image.resize((64, 64), Image.LANCZOS)
    return image

#directory salvataggio immagini
original_dir = "/kaggle/working/original"
gan_dir = "/kaggle/working/gan"
stable_dir = "/kaggle/working/stable"

classifier_256 = SVC(kernel='linear')  # Usa tutte le CPU disponibili
classifier_64 = SVC(kernel='linear')

features_list_256 = []
features_list_64 = []
labels_256 = []
labels_64 = []

# Estrai immagini da COCO
coco_images = os.listdir(coco_root)

#estrai immagini da gan
gan_images = os.listdir(gan_root)

#estrai immagini da stable
stable_images = os.listdir(stable_root)

step = 0
original = True
gan = True
stable = True

#Append coco's image
print("sei in coco")
for coco_img_name in coco_images:
    img_path = os.path.join(coco_root, coco_img_name)
    try:
        # Immagine reale
        img = Image.open(img_path).convert("RGB")
        image_real_256 = downsample_to_256(img)
        image_real_64 = downsample_to_64(img)
        if(original == True):
            save_image_real(img, original_dir, coco_img_name)
            save_image_256(image_real_256, original_dir, coco_img_name)
            save_image_64(image_real_64, original_dir, coco_img_name)
            original = False
            
        real_features_256 = extract_vgg_features(image_real_256, vgg16)
        real_features_64 = extract_vgg_features(image_real_64, vgg16)
        
        features_list_256.append(real_features_256)
        features_list_64.append(real_features_64)
        
        labels_256.append(1)  # etichetta per le immagini reali
        labels_64.append(1)
        step += 1

    except Exception as e:
        print(f"Errore nell'elaborazione dell'immagine ID {coco_img_name}: {e}")
        
#Append stable's image
step = 0
print("sei in stable")
for stable_img_name in stable_images:
    img_path = os.path.join(stable_root, stable_img_name)

    try:
        # Immagine da Stable Diffusion
        img = Image.open(img_path).convert("RGB")
        image_stable_256 = downsample_to_256(img)
        image_stable_64 = downsample_to_64(img)
        if(stable == True):
            save_image_fake(img, stable_dir, stable_img_name)
            save_image_256(image_stable_256, stable_dir, stable_img_name)
            save_image_64(image_stable_64, stable_dir, stable_img_name)
            stable = False
            
        stable_features_256 = extract_vgg_features(image_stable_256, vgg16)
        stable_features_64 = extract_vgg_features(image_stable_64, vgg16)
        
        features_list_256.append(stable_features_256)
        features_list_64.append(stable_features_64)
        
        labels_256.append(0)  # etichetta per le immagini reali
        labels_64.append(0)
        step += 1
    except Exception as e:
        print(f"Errore nell'elaborazione dell'immagine Stable {stable_img_name}: {e}")


# Elaborazione delle immagini GAN
step = 0
print("sei in gan")
for gan_img_name in gan_images:
    img_path = os.path.join(gan_root, gan_img_name)

    try:
        # Immagine da Stable Diffusion
        img = Image.open(img_path).convert("RGB")
        image_gan_256 = upsample_to_256(img)
        if(gan == True):
            save_image_fake(img, gan_dir, gan_img_name)
            save_image_256(image_gan_256, gan_dir, gan_img_name)
            save_image_64(img, gan_dir, gan_img_name)
            gan = False
        gan_features_256 = extract_vgg_features(image_gan_256, vgg16)
        gan_features_64 = extract_vgg_features(img, vgg16)
        
        features_list_256.append(gan_features_256)
        features_list_64.append(gan_features_64)
        
        labels_256.append(0)  # etichetta per le immagini reali
        labels_64.append(0)
        step += 1
    except Exception as e:
        print(f"Errore nell'elaborazione dell'immagine Gan {gan_img_name}: {e}")


# Addestra il classificatore SVM
classifier_256.fit(features_list_256, labels_256)
classifier_64.fit(features_list_256, labels_64)

# Salva il modello finale
joblib.dump(classifier_256, 'svm_classifier_256.pkl')
print("Modello SVM salvato come 'svm_classifier_256.pkl'")
joblib.dump(classifier_64, 'svm_classifier_64.pkl')

sei in coco
Immagine salvata in: /kaggle/working/original/real_image_real_image_248242.jpg.jpg
Immagine salvata in: /kaggle/working/original/image_256x256real_image_248242.jpg.jpg
Immagine salvata in: /kaggle/working/original/image_64x64real_image_248242.jpg.jpg
sei in stable
Immagine salvata in: /kaggle/working/stable/fake_image_fake_image_356406.jpg.jpg
Immagine salvata in: /kaggle/working/stable/image_256x256fake_image_356406.jpg.jpg
Immagine salvata in: /kaggle/working/stable/image_64x64fake_image_356406.jpg.jpg
sei in gan
Immagine salvata in: /kaggle/working/gan/fake_image_generated_image_1260.png.jpg
Immagine salvata in: /kaggle/working/gan/image_256x256generated_image_1260.png.jpg
Immagine salvata in: /kaggle/working/gan/image_64x64generated_image_1260.png.jpg
Modello SVM salvato come 'svm_classifier_256.pkl'


['svm_classifier_64.pkl']

In [10]:
import shutil

# Specifica il percorso della cartella che vuoi comprimere
folder_to_zip = '/kaggle/working/stable'  # Sostituisci con il percorso della tua cartella
output_zip = '/kaggle/working/stable.zip'

# Crea un file ZIP della cartella
shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)

'/kaggle/working/stable.zip'